In [25]:
from SafeTransformer import SafeTransformer
from sklearn.base import BaseEstimator, TransformerMixin

import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

from copy import deepcopy

In [5]:
titanic = pd.read_csv("titanic_train.csv")
y = titanic["Survived"]
X = titanic.drop(["PassengerId", "Survived", "Name", "Ticket"], axis=1)

X["Embarked"] = X["Embarked"].astype('category')
X["Cabin"] = X["Cabin"].fillna("unknown")
X["Cabin"] = X["Cabin"].astype('category')
X["Sex"] = X["Sex"].astype('category')

# Filling NaNs

In [9]:
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])

In [10]:
fill_NaN_pipeline = make_pipeline(
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            TypeSelector(np.number),
            SimpleImputer(strategy="median"),
        )),
        ("categorical_features", make_pipeline(
            TypeSelector("category"),
            SimpleImputer(strategy="most_frequent"),
        ))
    ])
)

In [13]:
X_filled = fill_NaN_pipeline.fit_transform(X)
ts_category = TypeSelector("category")
colnames_category = ts_category.fit_transform(X).columns
ts_numeric = TypeSelector(np.number)
colnames_numeric = ts_numeric.fit_transform(X).columns
colnames = colnames_numeric.append(colnames_category)
X_filled = pd.DataFrame(X_filled, columns=colnames)
X_filled.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Cabin,Embarked
0,3,22,1,0,7.25,male,unknown,S
1,1,38,1,0,71.2833,female,C85,C
2,3,26,0,0,7.925,female,unknown,S
3,1,35,1,0,53.1,female,C123,S
4,3,35,0,0,8.05,male,unknown,S


In [14]:
for col in X.columns:
    X_filled[col] = X_filled[col].astype(X[col].dtype)
X_filled.dtypes

Pclass         int64
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Sex         category
Cabin       category
Embarked    category
dtype: object

# Data set with dummy variables and splits


In [15]:
X_dummy = X_filled.copy()
colnames = list(X_dummy)
for idx, name in enumerate(colnames):
    if str(X_dummy.loc[:, name].dtype) in ['category', 'object']:
        dummies = pd.get_dummies(X_dummy.loc[:, name], prefix=name, drop_first=True)
        dummy_index  = X_dummy.columns.get_loc(name)
        X_dummy = pd.concat([X_dummy.iloc[:,range(dummy_index)], dummies, X_dummy.iloc[:, range(dummy_index+1, len(X_dummy.columns))]], axis=1)

In [16]:
X_train, X_test, X_lin_train, X_lin_test, y_train, y_test = train_test_split(X_filled, X_dummy, y, random_state = 123)

# SAFE

logistic regression

In [22]:
logistic_model = LogisticRegression(solver='liblinear')
logistic_model = logistic_model.fit(X_lin_train, y_train)
standard_predictions = logistic_model.predict(X_lin_test)
print(accuracy_score(y_test, standard_predictions))

0.7982062780269058


surrogate model

In [19]:
surrogate_model = XGBClassifier(eval_metric = 'auc', random_state = 123,
                               subsample = 0.8, n_estimators = 200, min_child_weight = 3, 
                                max_depth = 6, learning_rate = 0.4, reg_lambda = 0.8, 
                                gamma = 1.5, colsample_bytree = 0.8)
surrogate_model = surrogate_model.fit(X_lin_train, y_train)
surrogate_model_predictions = surrogate_model.predict(X_lin_test)
print(accuracy_score(y_test, surrogate_model_predictions))

0.8475336322869955


white-box model (logistic regression)

In [23]:
pens = range(1, 10)
best_score = float('-Inf')
best_pen = 0
aucs = []

for pen in pens:
    surrogate_model = XGBClassifier(eval_metric = 'auc', random_state = 123,
                               subsample = 0.8, n_estimators = 200, min_child_weight = 3, 
                                max_depth = 6, learning_rate = 0.4, reg_lambda = 0.8, 
                                gamma = 1.5, colsample_bytree = 0.8)
    logistic_model_simple = LogisticRegression(solver='liblinear')
    safe_transformer = SafeTransformer(surrogate_model, penalty = pen)
    pipe = Pipeline(steps=[('safe', safe_transformer), ('linear', logistic_model_simple)])
    pipe = pipe.fit(X_train, y_train)
    predictions = pipe.predict(X_test)
    auc = accuracy_score(y_test, predictions)
    aucs.append(auc)
    print([pen,auc])
    if auc > best_score:
        best_transformer = deepcopy(safe_transformer)
        best_model = deepcopy(logistic_model_simple)
        best_score = auc
        best_pen = pen

[1, 0.8251121076233184]
[2, 0.820627802690583]
[3, 0.820627802690583]
[4, 0.8071748878923767]
[5, 0.8026905829596412]
[6, 0.8026905829596412]
[7, 0.8026905829596412]
[8, 0.8026905829596412]
[9, 0.8026905829596412]


In [24]:
best_score

0.8251121076233184

## Plot